# WaDE Water Right Ex: Anlytics on Utah Domestic Water Rights
#### By Adel Abdallah and Ryan James - Western States Water Council
- Date Update: 11/20/2023
- WaDE API documentation: https://api.westernstateswater.org/
- Example API return for Water Right data: https://wade-api.azure-api.net/v1/SiteAllocationAmounts?BeneficialUseCV=Agriculture%20Irrigation&StartIndex=1&key=beba8a9dd8724fabb3b16d2a415e9aab

Notes:
- Our API only returns 1000 records at a time. We use a loop & append to dataframe approach here.

In [ ]:
# Needed Libararies
!pip install numpy
!pip install pandas
!pip install openpyxl
import openpyxl
import os
import numpy as np
import pandas as pd
import requests
import json

pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook
print ("Libararies have been imported")

In [ ]:
# Setting work directory
cwd = os.getcwd()
Output = cwd
print("current directory = "+Output)

## Retrieve Data using WaDE 2.0 SiteAllocationAmounts API.
- performing a loop with StartIndex = 0, then do iterations of 1000 rows.
- Default return is a 1000 rows per API call.  Can change with RecordCount element.

In [ ]:
# Create output dataframes to store each dictionary section of the API return.
Organizations = pd.DataFrame()
WaterSources = pd.DataFrame()
VariableSpecifics = pd.DataFrame()
Methods = pd.DataFrame()
BeneficialUses = pd.DataFrame()
WaterAllocations = pd.DataFrame()
Sites = pd.DataFrame()
print("done")

In [ ]:
# Pre-set base elements of API call for easy construction.
# Save as strings.
# uat connection

# base API URL.
base_url_API = "https://wade-api.azure-api.net/v1/SiteAllocationAmounts?"

# set a benefical use pararemter as "Agriculture Irrigation" (%20 = " ", blank splace)
benUseParam = "BeneficialUseCV=Domestic"

# limit search to state of interest, use abbreviation.
stateParam = "State=UT"

# security API Key.
keyParam = "key=beba8a9dd8724fabb3b16d2a415e9aab"

print("done")

In [ ]:
%%time 
# loop over the API calls through pages (index) one-at-a-time. 

startIndex = 0  # will do bounds of 1000
loopsToRun = 9 # number of loops we want to run


# The loop
loopCounter = 0  # counter of the number of loops
while loopCounter < loopsToRun:
    print(f'Loop #: ', loopCounter)

    startIndexParam = "StartIndex=" + str(startIndex)
    print (startIndexParam)
    
    # combine the API parameters together
    callString = base_url_API + benUseParam + "&" + stateParam + "&" + startIndexParam + "&" + keyParam
    print (callString)
    
    # Call the API, check if API has a response
    # The get
    try:
        response_dict = requests.get(callString).json()
        
        # Indexing and list slicing to append to individual tables.
        # Organizations Data       
        o_json_data = response_dict['Organizations'][0]
        o_tempDF = pd.json_normalize(o_json_data)
        o_tempDF = pd.DataFrame(o_tempDF, columns=['OrganizationName',
                                                        'OrganizationPurview',
                                                        'OrganizationWebsite',
                                                        'OrganizationState',
                                                        'OrganizationContactEmail',
                                                        'OrganizationPhoneNumber',
                                                        'OrganizationContactName',
                                                        'OrganizationContactEmail'], index=[0])
        Organizations = pd.concat([Organizations, o_tempDF])

        # WaterSource Data
        ws_json_data = response_dict['Organizations'][0]['WaterSources']
        ws_tempDF = pd.json_normalize(ws_json_data)
        WaterSources = pd.concat([WaterSources, ws_tempDF])
        
        # VariableSpecifics Data
        v_json_data = response_dict['Organizations'][0]['VariableSpecifics']
        v_tempDF = pd.json_normalize(v_json_data)
        VariableSpecifics = pd.concat([VariableSpecifics, v_tempDF])
         
        # Methods Data
        m_json_data = response_dict['Organizations'][0]['Methods']
        m_tempDF = pd.json_normalize(m_json_data)
        Methods = pd.concat([Methods, m_tempDF])
        
        # BeneficialUses Data
        bu_json_data = response_dict['Organizations'][0]['BeneficialUses']
        bu_tempDF = pd.json_normalize(bu_json_data)
        BeneficialUses = pd.concat([BeneficialUses, bu_tempDF])
        
        # WaterAllocations Data
        wa_json_data = response_dict['Organizations'][0]['WaterAllocations']
        wa_tempDF = pd.json_normalize(wa_json_data)
        wa_tempDF['StartIndex'] = str(startIndex) #tracking StartIndex used
        WaterAllocations = pd.concat([WaterAllocations, wa_tempDF])
        
        # Sites Data
        s_json_data = response_dict['Organizations'][0]['Sites']
        s_tempDF = pd.json_normalize(s_json_data)
        Sites = pd.concat([Sites, s_tempDF])
        
    except:
        print("StartIndex_param has no data")
          
            
    startIndex = startIndex + 1000
    loopCounter = loopCounter + 1
    
    print("------------")

    
# Remove duplicate based on unique UUID values from WaterSources, Variables, Methods, Beneficialuses, & Sites
WaterSources = WaterSources.drop_duplicates().sort_values(by=['WaterSourceUUID']).reset_index(drop=True)
VariableSpecifics = VariableSpecifics.drop_duplicates().reset_index(drop=True)
Methods = Methods.drop_duplicates().sort_values(by=['MethodUUID']).reset_index(drop=True)
BeneficialUses = BeneficialUses.drop_duplicates().sort_values(by=['Term']).reset_index(drop=True)
Sites = Sites.drop_duplicates(subset=['SiteUUID']).sort_values(by=['SiteUUID']).reset_index(drop=True)

print("done")

## Retrieve related POUs & PODS site info from sites dataframe, create new dataframe
- Creating easy to ready table for related POD sites per POU site from *RelatedPODSites* and *RelatedPOUSites* fields.
- Create temp POU table, create temp POD table, then concatenate into single output table.

In [ ]:
# Create new POU specific dataframe, populate with items from Sites dataframe
RelatedSitesPOU = pd.DataFrame()
RelatedSitesPOU['SourceSiteUUID'] = Sites['SiteUUID']
RelatedSitesPOU['SourcePODorPOU'] = Sites['PODorPOUSite']
RelatedSitesPOU['RelatedPODSites'] = Sites['RelatedPODSites']
RelatedSitesPOU = RelatedSitesPOU[RelatedSitesPOU['SourcePODorPOU'] == 'POU'].reset_index(drop=True)

try:
    # explode the entries in the RelatedPODSites column, reset index.
    RelatedSitesPOU = RelatedSitesPOU.assign(RelatedPODSites=RelatedSitesPOU['RelatedPODSites']).explode('RelatedPODSites').reset_index(drop=True)

    # fetch column RelatedPODSites as a Series, then return dataframe where the column labels are the keys of the dictionaries.
    RelatedSitesPOU = pd.concat([RelatedSitesPOU, RelatedSitesPOU["RelatedPODSites"].apply(pd.Series)], axis=1)

    # rename columns
    RelatedSitesPOU = RelatedSitesPOU.rename(columns={"SiteUUID": "RelatedSiteUUID"})

    # drop index & RelatedPODSites columns from dataframe (no longer needed).
    RelatedSitesPOU = RelatedSitesPOU.drop(columns=["RelatedPODSites", 0])
    
except:
    print("No POU sites for the returend SiteVariableAmounts records.")
    RelatedSitesPOU = pd.DataFrame()
    RelatedSitesPOU = RelatedSitesPOU.append(pd.Series(), ignore_index=True)


# drop null rows
RelatedSitesPOU = RelatedSitesPOU.dropna()

print(len(RelatedSitesPOU))
RelatedSitesPOU.head(1)

In [ ]:
# Create new POD specific dataframe, populate with items from Sites dataframe
RelatedSitesPOD = pd.DataFrame()
RelatedSitesPOD['SourceSiteUUID'] = Sites['SiteUUID']
RelatedSitesPOD['SourcePODorPOU'] = Sites['PODorPOUSite']
RelatedSitesPOD['RelatedPOUSites'] = Sites['RelatedPOUSites']
RelatedSitesPOD = RelatedSitesPOD[RelatedSitesPOD['SourcePODorPOU'] == 'POD'].reset_index(drop=True)

try: 
    # explode the entries in the RelatedPOUSites column, reset index.
    RelatedSitesPOD = RelatedSitesPOD.assign(RelatedPOUSites=RelatedSitesPOD['RelatedPOUSites']).explode('RelatedPOUSites').reset_index(drop=True)

    # fetch column RelatedPOUSites as a Series, then return dataframe where the column labels are the keys of the dictionaries.
    RelatedSitesPOD = pd.concat([RelatedSitesPOD, RelatedSitesPOD["RelatedPOUSites"].apply(pd.Series)], axis=1)

    # rename columns
    RelatedSitesPOD = RelatedSitesPOD.rename(columns={"SiteUUID": "RelatedSiteUUID"})

    # drop index & RelatedPOUSites columns from dataframe (no longer needed).
    RelatedSitesPOD = RelatedSitesPOD.drop(columns=["RelatedPOUSites", 0])
    
except:
    print("No POD sites for the returend SiteVariableAmounts records.")
    RelatedSitesPOD = pd.DataFrame()
    RelatedSitesPOD = RelatedSitesPOD.append(pd.Series(), ignore_index=True)
    

# drop null rows
RelatedSitesPOD = RelatedSitesPOD.dropna()

print(len(RelatedSitesPOD))
RelatedSitesPOD.head(1)

In [ ]:
# Concatenate POU with POD data
try:
    frames = [RelatedSitesPOU, RelatedSitesPOD]
    RelatedSites = pd.concat(frames).reset_index(drop=True)
    RelatedSites = RelatedSites.sort_values(by=['SourcePODorPOU', 'SourceSiteUUID', 'RelatedSiteUUID', 'StartDate', 'EndDate'])
    print(len(RelatedSites))
    RelatedSites.head(1)
except:
    print("No POD to POU relations in the data.")
    RelatedSites = pd.DataFrame()

## Export results
- Create a Pandas Excel writer, save each dataframe to a separate sheet.

In [ ]:
with pd.ExcelWriter('results/WaDE_API_Ex_Results.xlsx') as writer:
    Organizations.to_excel(writer, sheet_name='Organizations')
    WaterSources.to_excel(writer, sheet_name='WaterSources')
    VariableSpecifics.to_excel(writer, sheet_name='VariableSpecifics')
    Methods.to_excel(writer, sheet_name='Methods')
    BeneficialUses.to_excel(writer, sheet_name='BeneficialUses')
    WaterAllocations.to_excel(writer, sheet_name='WaterAllocations')
    Sites.to_excel(writer, sheet_name='Sites')
    RelatedSites.to_excel(writer, sheet_name='RelatedSites')

print("done")

# Analytics and Figures

In [ ]:
# Needed Libraries / Modules

# ---- working with data ----
import os  # native operating system interaction
import numpy as np  # mathematical array manipulation
import pandas as pd  # data structure and data analysis
import geopandas as gpd  # geo-data structure and data analysis

# ---- visualization ----
import geoplot as gplt  # for plotting maps and geo-data
import geoplot.crs as gcrs  #used to pull in webdata related to maps and geo-data
import missingno as msno # creates a matrix chart to show missing values
import plotly.express as px
import plotly.graph_objects as go  # for subplot creation
from plotly.subplots import make_subplots  # for subplot creation
import matplotlib.pyplot as mplt  # use with gplt to save fig to pdf

# ---- cleanup ----
import re # string regular expression manipulation
from datetime import datetime # date and time manipulation
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook
pd.set_option('display.float_format', lambda x: '%.5f' % x) # suppress scientific notation in Pandas

In [ ]:
# ---- merge watersource.csv to sites.csv ----

# explode site.csv on WaterSourceUUIDs
dfstemp = Sites.copy()
dfstemp = dfstemp.explode('WaterSourceUUIDs').reset_index(drop=True) # split WaterSourceUUIDs 

# merge
dfstemp_ws = pd.merge(dfstemp, WaterSources[['WaterSourceUUID', 'WaterSourceTypeCV']], left_on='WaterSourceUUIDs', right_on='WaterSourceUUID', how='left')

# groupby site-watersource.csv via SiteUUID
dfstemp_ws = dfstemp_ws.groupby('SiteUUID').agg(lambda x: ','.join([str(elem) for elem in (list(set(x))) if elem != ""])).replace(np.nan, "").reset_index()

print(len(dfstemp_ws))
dfstemp_ws.head(1)

In [ ]:
# WaterAllocations dataframe
dfaa = WaterAllocations.copy()
dfaa['BeneficialUses'] = [','.join(map(str, l)) for l in dfaa['BeneficialUses']] # convert from list to comma separated string
dfaa['SitesUUIDs'] = [','.join(map(str, l)) for l in dfaa['SitesUUIDs']] # convert from list to comma separated string
dfaa['AllocationPriorityDate'] = pd.to_datetime(dfaa['AllocationPriorityDate'], errors = 'coerce') # format to date value
dfaa['AllocationPriorityDate'] = pd.to_datetime(dfaa["AllocationPriorityDate"].dt.strftime('%m/%d/%Y')) # format data value to YYYY-MM-DD

In [ ]:
# ---- Histogram: Num of POD sites vs POU sites ----
print(dfstemp_ws.PODorPOUSite.value_counts())

fig = px.histogram(dfstemp_ws, x="PODorPOUSite")
fig.update_layout(hovermode=False,
                  bargap=0.2,
                  title="Histogram of PODorPOUSite Entries in sites.csv",
                  xaxis_title="PODorPOUSite Value",
                  yaxis_title="# of entries",
                  #legend_title="Legend Title",
                    font=dict(
                        family="Arial Bold",
                        size=12,
                        color="Black")
                 )
fig.show()
fig.write_image('figures/PODorPOUSite.png', engine="kaleido")

In [ ]:
# ---- Histogram: Num of sites via WatersourceTypeCV ----
print(dfstemp_ws.WaterSourceTypeCV.value_counts())

fig = px.histogram(dfstemp_ws, x="WaterSourceTypeCV")
fig.update_layout(bargap=0.2,
                  title="Histogram of WaterSourceTypeCV Entries in sites.csv",
                  xaxis_title="WaterSourceTypeCV Value",
                  yaxis_title="# of entries",
                  #legend_title="Legend Title",
                    font=dict(
                        family="Arial Bold",
                        size=12,
                        color="Black")
                 )
fig.show()
fig.write_image('figures/WaterSourceTypeCV.png', engine="kaleido")

In [ ]:
# ---- Histogram: Distribution of PrimaryBeneficialUseCategory WaDE Values ----
print(dfaa.BeneficialUses.value_counts())

fig = px.histogram(dfaa, x="BeneficialUses")
fig.update_layout(bargap=0.2,
                  title="Histogram of WaDE BeneficialUses Entries in waterallocations.csv",
                  xaxis_title="BeneficialUses Value",
                  yaxis_title="# of entries",
                  #legend_title="Legend Title",
                    font=dict(
                        family="Arial Bold",
                        size=12,
                        color="Black")
                 )
fig.show()
fig.write_image('figures/BeneficialUses.png', engine="kaleido")

In [ ]:
# ---- AllocationPriorityDate #1: histogram distribution of WaDE values 
print(dfaa.AllocationPriorityDate.value_counts())

dfaatemp = dfaa.copy()
dfaatemp = dfaatemp[(dfaatemp['ExemptOfVolumeFlowPriority'] < 1)].reset_index(drop=True)
fig = px.histogram(dfaatemp, x="AllocationPriorityDate")
fig.update_layout(bargap=0.2,
                  title="Histogram of AllocationPriorityDate Entries in waterallocations.csv",
                  xaxis_title="AllocationPriorityDate Value",
                  yaxis_title="# of entries",
                  #legend_title="Legend Title",
                    font=dict(
                        family="Arial Bold",
                        size=12,
                        color="Black")
                 )
fig.show()
fig.write_image('figures/AllocationPriorityDate1.png', engine="kaleido")

In [ ]:
# ---- AllocationPriorityDate #2: cumulative distribution of WaDE values 
print(dfaa.AllocationPriorityDate.value_counts())

dfaatemp = dfaa.copy()
dfaatemp = dfaatemp[(dfaatemp['ExemptOfVolumeFlowPriority'] < 1)].reset_index(drop=True)
fig = px.ecdf(dfaatemp, x="AllocationPriorityDate", ecdfnorm=None)
fig.update_layout(bargap=0.2,
                  title="Cumulative Distribution of AllocationPriorityDate Entries in waterallocations.csv",
                  xaxis_title="AllocationPriorityDate Value",
                  xaxis=dict(tickformat="%Y-%M"),
                  yaxis_title="# of entries",
                  #legend_title="Legend Title",
                    font=dict(
                        family="Arial Bold",
                        size=12,
                        color="Black")
                 )

fig.show()
fig.write_image('figures/AllocationPriorityDate2.png', engine="kaleido")

In [ ]:
# ---- AllocationLegalStatusCodeCV: histogram distribution of WaDE values ----
print(dfaa.AllocationLegalStatusCodeCV.value_counts())

fig = px.histogram(dfaa, x="AllocationLegalStatusCodeCV")
fig.update_layout(bargap=0.2,
                  title="Histogram of AllocationLegalStatusCodeCV Entries in waterallocations.csv",
                  xaxis_title="AllocationLegalStatusCodeCV Value",
                  yaxis_title="# of entries",
                  #legend_title="Legend Title",
                    font=dict(
                        family="Arial Bold",
                        size=12,
                        color="Black")
                 )
fig.show()
fig.write_image('figures/AllocationLegalStatusCodeCV.png', engine="kaleido")

In [ ]:
# ---- AllocationFlow_CFS: Boxplot distribution of WaDE values ----

try: 
    trace1 = go.Violin(x=dfaa['AllocationFlow_CFS'], points='outliers', name='Violin Plot')
    trace2 = go.Histogram(x=dfaa['AllocationFlow_CFS'], name='Historgram')

    fig = make_subplots(rows=2, cols=1)
    fig.add_trace(trace1, row=1, col=1)
    fig.add_trace(trace2, row=2, col=1)

    fig.update_layout(showlegend=False, bargap=0.2, title="AllocationFlow_CFS Distribution in waterallocations.csv", font=dict(family="Arial Bold", size=12,color="Black"))
    fig.update_xaxes(title_text="AllocationFlow_CFS Value", row=1, col=1)
    fig.update_xaxes(title_text="AllocationFlow_CFS Value", row=2, col=1)
    fig.update_yaxes(title_text="Num. of Records", row=2, col=1)
    fig.show()
    fig.write_image('figures/AllocationFlow_CFS.png', engine="kaleido")

except: print('Could not plot AllocationFlow_CFS value.')

In [ ]:
# ---- AllocationVolume_AF: Boxplot distribution of WaDE values ----

try:
    trace1 = go.Violin(x=dfaa['AllocationVolume_AF'], points='outliers', name='Violin Plot')
    trace2 = go.Histogram(x=dfaa['AllocationVolume_AF'], name='Historgram')

    fig = make_subplots(rows=2, cols=1)
    fig.add_trace(trace1, row=1, col=1)
    fig.add_trace(trace2, row=2, col=1)

    fig.update_layout(showlegend=False, bargap=0.2, title="AllocationVolume_AF Distribution in waterallocations.csv", font=dict(family="Arial Bold", size=12,color="Black"))
    fig.update_xaxes(title_text="AllocationVolume_AF Value", row=1, col=1)
    fig.update_xaxes(title_text="AllocationVolume_AF Value", row=2, col=1)
    fig.update_yaxes(title_text="Num. of Records", row=2, col=1)
    fig.show()
    fig.write_image('figures/AllocationVolume_AF.png', engine="kaleido")

except: print('Could not plot AllocationVolume_AF value.')

In [ ]:
# ---- Map of Points sites ----

dfstemp = Sites.copy()
dfstemp = dfstemp[dfstemp['PODorPOUSite'] == 'POD'].reset_index(drop=True)

try:
    contiguous_usa = gpd.read_file(gplt.datasets.get_path('contiguous_usa')) # use for background map in subplot
    ax = gplt.webmap(contiguous_usa, projection=gcrs.WebMercator()) # set subplot
    gdfstemp = gpd.GeoDataFrame(dfstemp, geometry=gpd.points_from_xy(dfstemp.Longitude.astype(float), dfstemp.Latitude.astype(float)), crs="EPSG:4326")
    gplt.pointplot(gdfstemp, hue='PODorPOUSite', legend=True, legend_var='hue', ax=ax)
    mplt.savefig(format="png", fname='figures/PointMap.png') 
except:
    print('No point data to plot')

In [ ]:
# ---- Map of Polygons ----

dfstemp = Sites.copy()
dfstemp = dfstemp[dfstemp['PODorPOUSite'] == 'POU'].reset_index(drop=True)

try:
    contiguous_usa = gpd.read_file(gplt.datasets.get_path('contiguous_usa')) # use for background map in subplot
    ax = gplt.webmap(contiguous_usa, projection=gcrs.WebMercator()) # set subplot

    dfstemp['Geometry'] = gpd.GeoSeries.from_wkt(dfstemp['SiteGeometry'], crs="EPSG:4326")
    gdfstemp = gpd.GeoDataFrame(dfstemp, geometry=dfstemp['Geometry'], crs="EPSG:4326") # covert to geodataframe
    gplt.polyplot(gdfstemp, ax=ax)
    mplt.savefig(format="png", fname='figures/PolyMap.png')
except:
    print('No geometry data to plot')